### Intro

This note book will organize the galaxy catalogs from the UniverseMachine model using the small multidark planck simulation (SMDPL).  The results are provided by Peter Behroozi, and the initial catalogs are made by Christopher Bradshaw. 

The models are for galaxies at z~0.4.  Two models are provided: 
1. `sfr_catalog_0.712400.txt.gz`: old fashion `bcg` and `icl` components one. 
2. `sfr_catalog_insitu_exsitu_0.712400.txt.gz`: new `in-situ` and `ex-situ` components one. 
3. `particles_0.712400.txt.gz`: DM particles file. 

The useful information has been summarized into a smaller catalog, also the central and satellite galaxies are separated now.  (`.npz`) format. 

Here, we: 

1. Explore the basic properties of these new models. 
2. Compare their SMF with observed ones. 
3. Explore new M100-M10 models.
4. Select the relevant massive galaxies, and save the value-added catalog. 

We also need to downsample the particle tables and pre-compute the `DeltaSigma` signals around the selected massive galaxies.

#### Small MultiDark Simulation

* For details, please see: https://www.cosmosim.org/cms/simulations/smdpl/

![](cosmosim_smdpl_sum.png)

In [1]:
%matplotlib inline
%load_ext autoreload 
%autoreload 2

import os
import sys
import copy 
from time import time

import numpy as np 
import matplotlib.pyplot as plt
    
import matplotlib.mlab as ml
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MaxNLocator

%config InlineBackend.figure_format='retina'
plt.rcParams['figure.dpi'] = 100.0
plt.rcParams['figure.facecolor'] = 'w'
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 12.0
plt.rc('text', usetex=True)

# Personal repo
from stellar_mass_function import compute_smf, \
    bootstrap_smf
from load_mdpl2_mock import value_added_smdpl_mock
from model_predictions import total_stellar_mass_including_satellites

from prepare_universe_machine import prep_um_catalog
    
from hsc_massive import \
    s16a_path, \
    sample_selection, \
    prepare_sed, \
    catalog_summary, \
    smhm, \
    plotting

/usr/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from astropy.table import Table, Column, vstack

# Summary catalogs for SMDPL-UniverseMachine model

In [3]:
# Setup environment 
envir = s16a_path.set_env(version='astro5')
#envir = s16a_path.set_env(version='kungpao')

# Folder for new UM model 
um_dir = os.path.join(envir['s16a'], 'um2/um2_new')
um_smdpl_dir = os.path.join(um_dir, 'um_smdpl_small')

# Summary catalogs for new UM models 
um_smdpl_old = np.load(os.path.join(um_smdpl_dir, 
                                    'sfr_catalog_0.712400_final.npz'))
um_smdpl_new = np.load(os.path.join(um_smdpl_dir, 
                                    'sfr_catalog_insitu_exsitu_0.712400_final_extended.npz'))

# Results for central and satellite galaxies
um_smdpl_old_cen = Table(um_smdpl_old['centrals'])
um_smdpl_old_sat = Table(um_smdpl_old['satellites'])

um_smdpl_new_cen = Table(um_smdpl_new['centrals'])
um_smdpl_new_sat = Table(um_smdpl_new['satellites'])

print(len(um_smdpl_old_cen), len(um_smdpl_old_sat))
print(len(um_smdpl_new_cen), len(um_smdpl_new_sat))

print(um_smdpl_old_cen.colnames)

(385125, 10809469)
(385125, 10809469)
['id', 'upid', 'x', 'y', 'z', 'm', 'mp', 'sm', 'icl', 'sfr']


In [5]:
# vstack the centrals and satellites 
um_smdpl_old_all = vstack([um_smdpl_old_cen, um_smdpl_old_sat])
um_smdpl_new_all = vstack([um_smdpl_new_cen, um_smdpl_new_sat])

In [7]:
um_smdpl_new_all.rename_column('mvir', 'mvir_hlist')
um_smdpl_new_all.rename_column('rvir', 'rvir_hlist')
um_smdpl_new_all.rename_column('rs', 'rs_hlist')
um_smdpl_new_all.rename_column('Halfmass_Scale', 'scale_half_mass')
um_smdpl_new_all.rename_column('scale_of_last_MM', 'scale_last_mm')
um_smdpl_new_all.rename_column('M200b', 'm200b_hlist')
um_smdpl_new_all.rename_column('M200c', 'm200c_hlist')
um_smdpl_new_all.rename_column('Acc_Rate_Inst', 'acc_rate_inst')
um_smdpl_new_all.rename_column('Acc_Rate_100Myr', 'acc_rate_100myr')
um_smdpl_new_all.rename_column('Acc_Rate_1*Tdyn', 'acc_rate_1_tdyn')
um_smdpl_new_all.rename_column('Acc_Rate_2*Tdyn', 'acc_rate_2_tdyn')
um_smdpl_new_all.rename_column('Acc_Rate_Mpeak', 'acc_rate_mpeak')
um_smdpl_new_all.rename_column('Vmax@Mpeak', 'vmax_at_mpeak')

In [8]:
# Value added the catalogs 
um_smdpl_old_all = value_added_smdpl_mock(um_smdpl_old_all)
um_smdpl_new_all = value_added_smdpl_mock(um_smdpl_new_all)

In [9]:
um_smdpl_old_all_vagc = prep_um_catalog(um_smdpl_old_all, um_min_mvir=11.5)
um_smdpl_new_all_vagc = prep_um_catalog(um_smdpl_new_all, um_min_mvir=11.5)

print(um_smdpl_new_all_vagc.colnames)
print(len(um_smdpl_new_all_vagc))

['halo_id', 'upid', 'x', 'y', 'z', 'mvir', 'mpeak', 'sm', 'icl', 'sfr', 'mvir_hlist', 'rvir_hlist', 'rs_hlist', 'scale_half_mass', 'scale_last_mm', 'm200b_hlist', 'm200c_hlist', 'acc_rate_inst', 'acc_rate_100myr', 'acc_rate_1_tdyn', 'acc_rate_2_tdyn', 'acc_rate_mpeak', 'vmax_at_mpeak', 'halo_hostid', 'mhalo_host', 'mask_central', 'mtot_galaxy', 'mstar_mhalo', 'logms_gal', 'logms_icl', 'logms_tot', 'logms_halo', 'logmh_vir', 'logmh_peak', 'logmh_host']
11193763


In [10]:
# Save the results

# FITS table
um_smdpl_old_all_vagc.write(os.path.join(um_dir,
                                         'um_smdpl_0.7124_old_vagc_mhost_11.5.fits'),
                            format='fits', overwrite=True)
um_smdpl_new_all_vagc.write(os.path.join(um_dir,
                                         'um_smdpl_0.7124_new_vagc_mhost_11.5_extended.fits'),
                            format='fits', overwrite=True)

# Numpy array 
np.save(os.path.join(um_dir,
                     'um_smdpl_0.7124_old_vagc_mhost_11.5.npy'),
        np.asarray(um_smdpl_old_all_vagc)
       )
np.save(os.path.join(um_dir,
                     'um_smdpl_0.7124_new_vagc_mhost_11.5_extended.npy'),
        np.asarray(um_smdpl_new_all_vagc)
       )

## Select a sample of galaxies in massive haloes for our model

* Choose logMh_peak > 11.5 for now

In [11]:
um_smdpl_old_mvagc = um_smdpl_old_all[um_smdpl_old_all['logmh_peak'] >= 11.5]
um_smdpl_new_mvagc = um_smdpl_new_all[um_smdpl_new_all['logmh_peak'] >= 11.5]

print(len(um_smdpl_new_mvagc))

806306


In [12]:
# Save the results

# FITS table
um_smdpl_old_mvagc.write(os.path.join(um_dir,
                                      'um_smdpl_0.7124_old_vagc_mpeak_11.5.fits'),
                            format='fits', overwrite=True)
um_smdpl_new_mvagc.write(os.path.join(um_dir,
                                      'um_smdpl_0.7124_new_vagc_mpeak_11.5.fits'),
                            format='fits', overwrite=True)

# Numpy array 
np.save(os.path.join(um_dir,
                     'um_smdpl_0.7124_old_vagc_mpeak_11.5.npy'),
        np.asarray(um_smdpl_old_mvagc)
       )
np.save(os.path.join(um_dir,
                     'um_smdpl_0.7124_new_vagc_mpeak_11.5.npy'),
        np.asarray(um_smdpl_new_mvagc)
       )